# TEMPO_O3TOT_L2 Harmony Tests

### Environment setup:

This notebook assumes that it is being run in a local Python environment, configured using either `pyenv` or conda. Either can be used, but the dependencies will be installed via Pip. To install the required packages to run this notebook:

```bash
$ pip install -r requirements.txt
```

A `.netrc` file must also be located in the user's home directory.

#### Import Requirements:

In [1]:
from os.path import exists
from os import makedirs, replace
from os.path import exists, join as path_join
from shutil import rmtree
from harmony import Client, Environment, Collection, Request
from harmony.harmony import ProcessingFailedException

#### Set the Harmony Environment:

In [2]:
harmony_host_url = 'http://localhost:3000'

### Identify Harmony environment (for easier reference):

In [3]:
host_environment = {'http://localhost:3000': Environment.LOCAL,
                    'https://harmony.sit.earthdata.nasa.gov': Environment.SIT,
                    'https://harmony.uat.earthdata.nasa.gov': Environment.UAT,
                    'https://harmony.earthdata.nasa.gov': Environment.PROD}


harmony_environment = host_environment.get(harmony_host_url)

if harmony_environment is not None:
    harmony_client = Client(env=harmony_environment)

#### TEMPO_O3TOT_L2 Data:

In [4]:
collection = Collection(id='C1270257471-EEDTEST')
granule_id='G1270257472-EEDTEST'

#### Building Swath Projector Request:

In [5]:
harmony_client = Client(env=Environment.LOCAL)
request = Request(
  crs='EPSG:4326',
  interpolation='near',
  collection=collection,
  granule_id=granule_id
)

#### Create output directory to save test results:

In [6]:
collection_short_name = 'TEMPO_O3TOT_L2'
output_directory = path_join('output_files', collection_short_name) + '/'

if exists(output_directory):
    rmtree(output_directory)

makedirs(output_directory)

#### Submitting Harmony Request:

In [ ]:
def submit_and_download(harmony_client: Client, request: Request,
                        output_file_name: str):
    """ Submit a Harmony request via a `harmony-py` client. Wait for the
        Harmony job to finish, then download the results to the specified file
        path.

    """
    downloaded_filename = None

    try:
        job_id = harmony_client.submit(request)

        for filename in [file_future.result()
                         for file_future
                         in harmony_client.download_all(job_id,
                                                        overwrite=True)]:

            print(f'Downloaded: {filename}')
            downloaded_filename = filename

        if downloaded_filename is not None:
            replace(downloaded_filename, output_file_name)
            print(f'Saved output to: {output_file_name}')

    except ProcessingFailedException:
        print_error('Harmony request failed to complete successfully.')

def print_error(error_string: str) -> str:
    """Print an error, with formatting for red text. """
    print(f'\033[91m{error_string}\033[0m')

swath_output_file_name = output_directory + f'TEMPO_O3TOT_L2_{granule_id}_output.h5'
submit_and_download(harmony_client, request, swath_output_file_name)